In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score,
    precision_score,
    roc_auc_score,
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier, 
)
from lightgbm import LGBMClassifier  # LightGBM
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, Pool, cv  # CatBoost
from sklearn.ensemble import ExtraTreesClassifier  # Extra Trees
import xgboost as xgb  # XGBoost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.metrics import log_loss, roc_auc_score

# Initializing the LightGBM model with lower verbosity
lgbm_model = LGBMClassifier(verbosity=-1)  # -1 for minimal verbosity

# Initializing the CatBoost model with lower verbosity (0 to suppress output)
catboost_model = CatBoostClassifier(verbose=0)  # 0 to suppress output

import warnings

# Suppressing LightGBM and CatBoost warnings
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")
warnings.filterwarnings("ignore", category=UserWarning, module="catboost")
sns.set()
%matplotlib inline
import os
sns.set()

In [2]:
data = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/Project_10_Cupom reco/Data.csv')

In [3]:
df = data.copy()

In [4]:
pd.set_option('display.max_columns', 50)

In [5]:
# dropping irrelvant columns on basis of null values and dropping null rows, duplicated values
df = df.drop(columns='car')
df = df.dropna()
df = df.drop_duplicates(ignore_index=True)

In [6]:
# Renaming column

df = df.rename(columns = {'Accept(Y/N?)': 'Accept'})

In [7]:
# Creating new column 'Age' 
# (below21,21,26) -> young
# (31, 36, 41, 46) -> mid_age
# (above50) -> older

df.loc[df['age'] == 'below21', 'Age'] = 'young' 
df.loc[df['age'] == '21', 'Age'] = 'young' 
df.loc[df['age'] == '26', 'Age'] = 'young' 
df.loc[df['age'] == '31', 'Age'] = 'mid_age' 
df.loc[df['age'] == '36', 'Age'] = 'mid_age' 
df.loc[df['age'] == '41', 'Age'] = 'mid_age' 
df.loc[df['age'] == '46', 'Age'] = 'mid_age'
df.loc[df['age'] == '50plus', 'Age'] = 'older'

In [8]:
# Creating new column 'Income' 

# below 49999 - low income
# above 50000 - mid income

# we will not go for high income as High income individuals wont only be attracted by coupons and for these going to food places may vary on several other factors, can be for mid income level too but for now lets consider earlir

df.loc[df['income'] == 'Less than $12500','Income'] = 'Low'
df.loc[df['income'] == '$12500 - $24999', 'Income'] = 'Low'
df.loc[df['income'] == '$25000 - $37499','Income'] = 'Low'
df.loc[df['income'] == '$37500 - $49999', 'Income'] = 'Low'
df.loc[df['income'] == '$50000 - $62499', 'Income'] = 'Mid'
df.loc[df['income'] == '$62500 - $74999', 'Income'] = 'Mid'
df.loc[df['income'] == '$75000 - $87499','Income'] = 'Mid'
df.loc[df['income'] == '$87500 - $99999', 'Income'] = 'Mid'
df.loc[df['income'] == '$100000 or More', 'Income'] = 'Mid'

In [9]:
# we will create new feature based on education

# criteria High school(low_education)<college(low_education)<1st_degree(mid_education)<High_degree(high_education)

# low_education < mid_eduction < high_education

#Creating new feature by using logic statement for durantion required to travel
 
df.loc[df['education'] == 'Some High School', 'Level_of_education'] = 'low_education' 
df.loc[df['education'] == 'High School Graduate', 'Level_of_education'] = 'low_education' 
df.loc[df['education'] == 'Some college - no degree', 'Level_of_education'] = 'low_education' 
df.loc[df['education'] == 'Associates degree', 'Level_of_education'] = 'mid_education' 
df.loc[df['education'] == 'Bachelors degree', 'Level_of_education'] = 'mid_education' 
df.loc[df['education'] == 'Graduate degree (Masters or Doctorate)', 'Level_of_education'] = 'high_education' 

In [10]:
# creating new feature on basis of occupation and %acceptance of coupon

# criteria - =above 60% > High, 50-59.99% > Medium, below 49.99 > low acceptance

cross_tab = pd.crosstab(columns=[df['Accept']], index=[df['occupation']], margins=True, normalize='index')

# Calculate the acceptance percentage and categorize it
cross_tab['Acceptance%'] = cross_tab[1] * 100  # Calculate the acceptance percentage

# criteria for categorization
def categorize_acceptance(percentage):
    if percentage >= 60:
        return 'High'
    elif 50 <= percentage <= 59.99:
        return 'Medium'
    else:
        return 'Low'

# Applying the categorization function to the 'Acceptance%' column
cross_tab['Occupation_with_(1)%'] = cross_tab['Acceptance%'].apply(categorize_acceptance)

# Merging the 'Acceptance%' and 'Acceptance_Category' columns back into the 'df' DataFrame based on 'occupation'
df = df.merge(cross_tab[['Acceptance%', 'Occupation_with_(1)%']], left_on='occupation', right_index=True, how='left')

In [11]:
#Creating new feature by time required

df['Coupon_timeRequired_15to25'] = 0 
df.loc[(df['toCoupon_GEQ15min'] == 1) & (df['toCoupon_GEQ25min'] == 0), 'Coupon_timeRequired_15to25'] = 1
df.loc[df['toCoupon_GEQ15min'] == 0, 'Coupon_timeRequired_15to25'] = 0
df.loc[df['toCoupon_GEQ25min'] == 1, 'Coupon_timeRequired_15to25'] = 2

In [12]:
# we want to know if we offer a coupon to an individual does he accept it if he is the regular visitor to that category of place

Relative_coupon_Accepted_count = []

for i in range(df.shape[0]):
    if df['coupon'].iloc[i] == 'Restaurant(<20)':
        Relative_coupon_Accepted_count.append(df['RestaurantLessThan20'].iloc[i])
    elif df['coupon'].iloc[i] == 'Coffee House':
        Relative_coupon_Accepted_count.append(df['CoffeeHouse'].iloc[i])
    elif df['coupon'].iloc[i] == 'Carry out & Take away':
        Relative_coupon_Accepted_count.append(df['CarryAway'].iloc[i])
    elif df['coupon'].iloc[i] == 'Bar':
        Relative_coupon_Accepted_count.append(df['Bar'].iloc[i])
    elif df['coupon'].iloc[i] == 'Restaurant(20-50)':
        Relative_coupon_Accepted_count.append(df['Restaurant20To50'].iloc[i])

df['Relative_coupon_Accepted_count'] = Relative_coupon_Accepted_count

In [13]:
# dropping columns

df = df.drop(['Bar', 'CoffeeHouse', 'CarryAway',
       'RestaurantLessThan20', 'Restaurant20To50', 'age', 'income', 'education', 'occupation', 'Acceptance%', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_same', 'direction_opp', 'toCoupon_GEQ5min'], axis=1)

In [15]:
categorical_features = ['destination', 'passanger', 'weather', 'temperature', 'coupon',
       'expiration', 'gender', 'maritalStatus', 'has_children',
       'Age', 'Income', 'Level_of_education', 'Occupation_with_(1)%',
       'Coupon_timeRequired_15to25', 'Relative_coupon_Accepted_count']

In [16]:
# Apply label encoding to each categorical column
for col in categorical_features:
    df[col] = label_encoder.fit_transform(df[col])

In [17]:
y = df['Accept'].astype(int)  # Target variable
X = df.drop(columns=['Accept']).astype(int)  # Features

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2)

In [19]:
models = []
models.append(("LR", LogisticRegression()))
models.append(("LDA", LinearDiscriminantAnalysis()))
models.append(("KNN", KNeighborsClassifier()))
models.append(("CART", DecisionTreeClassifier()))
models.append(("NB", GaussianNB()))
models.append(("SVM", SVC()))
models.append(("RF", RandomForestClassifier()))
models.append(("ExtraTrees", ExtraTreesClassifier()))  # Extra Trees
models.append(("AdaBoost", AdaBoostClassifier()))  # AdaBoost
models.append(("XGBoost", xgb.XGBClassifier()))  # XGBoost

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s Accuracy: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

    # Calculating training accuracy
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Training Accuracy for {name}: {train_accuracy}")
    print()

    # Calculating test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy for {name}: {test_accuracy}")

    # Calculating additional metrics
    print(f"Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}")
    print()
    print(f"Classification Report for {name}:\n{classification_report(y_test, y_pred)}")
    print()
    print(f"Recall for {name}: {recall_score(y_test, y_pred)}")
    print()
    print(f"Precision for {name}: {precision_score(y_test, y_pred)}")
    print()
    print(f"ROC AUC for {name}: {roc_auc_score(y_test, y_pred)}")
    print("----------------------------------------------------------------------------")


LR Accuracy: 0.679029 (0.004855)
Training Accuracy for LR: 0.6797711137013882

Test Accuracy for LR: 0.661864406779661
Confusion Matrix for LR:
[[ 554  477]
 [ 321 1008]]

Classification Report for LR:
              precision    recall  f1-score   support

           0       0.63      0.54      0.58      1031
           1       0.68      0.76      0.72      1329

    accuracy                           0.66      2360
   macro avg       0.66      0.65      0.65      2360
weighted avg       0.66      0.66      0.66      2360


Recall for LR: 0.7584650112866818

Precision for LR: 0.6787878787878788

ROC AUC for LR: 0.6479036986598298
----------------------------------------------------------------------------
LDA Accuracy: 0.680089 (0.005066)
Training Accuracy for LDA: 0.68040690897531

Test Accuracy for LDA: 0.6610169491525424
Confusion Matrix for LDA:
[[ 560  471]
 [ 329 1000]]

Classification Report for LDA:
              precision    recall  f1-score   support

           0       0.63 

In [20]:
models = []
models.append(("LGBM", LGBMClassifier()))  # LightGBM

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s Accuracy: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

    # Calculating training accuracy
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Training Accuracy for {name}: {train_accuracy}")
    print()

    # Calculating test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy for {name}: {test_accuracy}")

    # Calculating additional metrics
    print(f"Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}")
    print()
    print(f"Classification Report for {name}:\n{classification_report(y_test, y_pred)}")
    print()
    print(f"Recall for {name}: {recall_score(y_test, y_pred)}")
    print()
    print(f"Precision for {name}: {precision_score(y_test, y_pred)}")
    print()
    print(f"ROC AUC for {name}: {roc_auc_score(y_test, y_pred)}")
    print("----------------------------------------------------------------------------")


[LightGBM] [Info] Number of positive: 4819, number of negative: 3674
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 48
[LightGBM] [Info] Number of data points in the train set: 8493, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.567408 -> initscore=0.271285
[LightGBM] [Info] Start training from score 0.271285
[LightGBM] [Info] Number of positive: 4820, number of negative: 3673
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000198 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 48
[LightGBM] [Info] Number of data points in the train set: 8493, number of used features: 15
[LightGBM] [Info] [binar

In [21]:
models = []
models.append(("CatBoost", CatBoostClassifier()))

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s Accuracy: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

    # Calculating training accuracy
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Training Accuracy for {name}: {train_accuracy}")
    print()

    # Calculating test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy for {name}: {test_accuracy}")

    # Calculating additional metrics
    print(f"Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}")
    print()
    print(f"Classification Report for {name}:\n{classification_report(y_test, y_pred)}")
    print()
    print(f"Recall for {name}: {recall_score(y_test, y_pred)}")
    print()
    print(f"Precision for {name}: {precision_score(y_test, y_pred)}")
    print()
    print(f"ROC AUC for {name}: {roc_auc_score(y_test, y_pred)}")
    print("----------------------------------------------------------------------------")


Learning rate set to 0.025683
0:	learn: 0.6875206	total: 147ms	remaining: 2m 26s
1:	learn: 0.6821093	total: 150ms	remaining: 1m 15s
2:	learn: 0.6771721	total: 154ms	remaining: 51.2s
3:	learn: 0.6719008	total: 158ms	remaining: 39.3s
4:	learn: 0.6669840	total: 161ms	remaining: 32.1s
5:	learn: 0.6623590	total: 165ms	remaining: 27.4s
6:	learn: 0.6579173	total: 169ms	remaining: 23.9s
7:	learn: 0.6538384	total: 172ms	remaining: 21.4s
8:	learn: 0.6502864	total: 176ms	remaining: 19.4s
9:	learn: 0.6466346	total: 180ms	remaining: 17.8s
10:	learn: 0.6432684	total: 184ms	remaining: 16.6s
11:	learn: 0.6399956	total: 188ms	remaining: 15.5s
12:	learn: 0.6365154	total: 191ms	remaining: 14.5s
13:	learn: 0.6333822	total: 194ms	remaining: 13.7s
14:	learn: 0.6303381	total: 197ms	remaining: 12.9s
15:	learn: 0.6280345	total: 200ms	remaining: 12.3s
16:	learn: 0.6252629	total: 202ms	remaining: 11.7s
17:	learn: 0.6227726	total: 206ms	remaining: 11.2s
18:	learn: 0.6205648	total: 210ms	remaining: 10.8s
19:	learn